In [5]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import glob
import www


masks = sorted(glob.glob("D:/ISIC2018_Task1_Training_GroundTruth_x2/*.png"))
files = sorted(glob.glob("D:/ISIC2018_Task1-2_Training_Input_x2/*.jpg"))

train_images, train_masks, val_masks, val_images, test_masks, test_images = www.split_data(files, masks, 0.2, 0.5)

print('Size of training set:', len(train_images))
print('Size of validation set:', len(val_images))
print('Size of test set:', len(test_images))

train_data = www.shuffle_map_data(train_images, train_masks)
val_data = www.shuffle_map_data(val_images, val_masks)
test_data = www.shuffle_map_data(test_images, test_masks)



Size of training set: 1816
Size of validation set: 389
2594
2594
2594
